In [ ]:
%pip install transformers
%pip install tensorflow

In [5]:
import pandas as pd
from transformers import BertTokenizer

root_path_gdrive = '/content/drive/MyDrive/Trabalho/analise_sent_repo/analise_sent/'

In [71]:
#train = pd.read_csv(root_path_gdrive + 'bases/train.tsv',sep='\t')
train = pd.read_csv(root_path_gdrive + 'bases/tweets_mg.csv')
train = train[['Text','Classificacao']]
train = train.rename(columns={'Text':'Phrase','Classificacao':'Sentiment'})

In [69]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
#tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/210k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [70]:
seq_len=512

In [72]:
tokens = tokenizer(train['Phrase'].tolist(), max_length=seq_len, truncation=True,
                   padding='max_length', add_special_tokens=True,
                   return_tensors='np')

In [73]:
labels = pd.get_dummies(train['Sentiment']).values
Xids = tokens['input_ids']
Xmask = tokens['attention_mask']

In [75]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [76]:
def map_func(input_ids, masks, labels):
    # converte input_ids e attention_mask em um dic (necessario para modelagem tf)
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

dataset = dataset.map(map_func)

dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int64, name=None)}, TensorSpec(shape=(3,), dtype=tf.uint8, name=None))>

In [77]:
batch_size = 16

dataset = dataset.shuffle(len(dataset)).batch(batch_size, drop_remainder=True)

dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 3), dtype=tf.uint8, name=None))>

In [78]:
split = 0.9

# Calcular quantos batches precisam ser considerados para temos um dataset de treino igual a 90%
size = int((Xids.shape[0] / batch_size) * split)

In [79]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)
del dataset

In [80]:
tf.data.experimental.save(train_ds, 'train')
tf.data.experimental.save(val_ds, 'val')

In [81]:
# dados necessarios para o tf.data.experimental.load, element_spec descreve o shape do tensor
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)},
 TensorSpec(shape=(16, 3), dtype=tf.uint8, name=None))

In [98]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

bert.summary()

Downloading:   0%|          | 0.00/529M [00:00<?, ?B/s]

Some layers from the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108923136 
                                                                 
Total params: 108,923,136
Trainable params: 108,923,136
Non-trainable params: 0
_________________________________________________________________


In [99]:
import tensorflow as tf

# duas camadas de entrada, garantimos que as variáveis ​​do nome da camada correspondam às chaves do dicionário no conjunto de dados TF
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

# acessamos o modelo do transformador dentro de nosso objeto bert usando o atributo bert (por exemplo, bert.bert em vez de bert)
embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # acessar ativações finais (já max-pooled) 
# converter bert embeddings em 5 classes de saída
x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
y = tf.keras.layers.Dense(3, activation='softmax', name='outputs')(x)

In [100]:
# inicializa modelo
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# congela bert layer
model.layers[2].trainable = False

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108923136   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                         

In [101]:
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [102]:
element_spec = ({'input_ids': tf.TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
            'attention_mask': tf.TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)},
            tf.TensorSpec(shape=(16, 3), dtype=tf.uint8, name=None))

# carrega training and validation sets
train_ds = tf.data.experimental.load('train', element_spec=element_spec)
val_ds = tf.data.experimental.load('val', element_spec=element_spec)

train_ds.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(16, 3), dtype=tf.uint8, name=None))>

In [103]:
history = model.fit(x= train_ds,validation_data=val_ds,epochs=3)

Epoch 1/3
461/461 [==============================] - 222s 463ms/step - loss: 0.3264 - accuracy: 0.8746 - val_loss: 0.2237 - val_accuracy: 0.9118
Epoch 2/3
461/461 [==============================] - 212s 459ms/step - loss: 0.2330 - accuracy: 0.9173 - val_loss: 0.1637 - val_accuracy: 0.9449
Epoch 3/3
461/461 [==============================] - 212s 459ms/step - loss: 0.1975 - accuracy: 0.9322 - val_loss: 0.1449 - val_accuracy: 0.9608


In [105]:
model.save(root_path_gdrive + 'sentiment_model')

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('sentiment_model')

model.summary()

In [106]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

def prep_data(text):
    tokens = tokenizer.encode_plus(text, max_length=512,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')
    return {'input_ids': tokens['input_ids'],'attention_mask': tokens['attention_mask']}

In [131]:
print(train.loc[7744,'Sentiment'], labels[7744]) # Positivo: 2
print(train.loc[4331,'Sentiment'], labels[4331]) # Neutro: 1
print(train.loc[3595,'Sentiment'], labels[3595]) # Negativo: 0

dic_sent = {2: 'Positivo', 1: 'Neutro', 0: 'Negativo'}

Positivo [0 0 1]
Neutro [0 1 0]
Negativo [1 0 0]


In [128]:
train.sample(10)

,Phrase,Sentiment
1533,'Jeitinho Kalil': novo prefeito de BH almoça d...,Positivo
678,"Com os dois pés na mierda, ou melhor, no atole...",Negativo
4460,"RT @UOLNoticias: Custaram R$ 21,8 milhões: Mes...",Negativo
4416,"RT @UOLNoticias: Custaram R$ 21,8 milhões: Mes...",Negativo
497,BB avisa Justiça que governo de MG desgovernad...,Negativo
4113,RT @rehventura: Eu queria ter a autoestima do ...,Neutro
7910,Dois homens são presos suspeitos de tráfico de...,Positivo
2355,RT @AnaPaulaVolei: Mais 2 helicópteros!!A cara...,Negativo
76,#ImpeachmentPimentel #PTNuncaMais \nMesmo em c...,Neutro
8180,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo


In [136]:
import numpy as np

probs = model.predict(prep_data('Quatro pessoas são presas acusadas de intolerância religiosa'))[0]

print(probs)
print(dic_sent[np.argmax(probs)])

[8.1236243e-07 1.7457695e-01 8.2542229e-01]
Positivo
